In [299]:
from ipstartup import *
import geopandas as gpd
import plotly.express as px
from mapboxer.basemap import *

[cellevents:43:INFO]:starting (time=Aug-06 16:50 MainProcess)


time: 2 ms


# uk districts

In [409]:
# job
job = pd.read_excel("../../registr/metadata/job.xlsx")
job = job[~job.jobtype.isin(["dev", "invalid"])]
job = job.groupby("authority").sum().reset_index()[["authority", "names"]]

# borders
borders = gpd.read_file(
    r"../..\registr\metadata\Local_Authority_Districts__December_2017__Boundaries_in_Great_Britain-shp\Local_Authority_Districts__December_2017__Boundaries_in_Great_Britain.shp"
)
# for 200 creates map in 3 seconds versus 1'30. likely loads faster too.
borders.geometry = borders.geometry.simplify(1000)
borders.geometry = borders.geometry.to_crs(epsg=4326)
borders = borders.rename(columns=dict(lad17nm="authority"))
borders.authority = borders.authority.str.replace("\'","")

# merged
df = borders.merge(job, on="authority", how="left")
df.loc[df.names>0, "registr"] = "user"
df.registr = df.registr.fillna("non-user")

# centroids for text labels
df_c = df.copy()
df_c.geometry = df_c.geometry.centroid

[cellevents:43:INFO]:starting (time=Aug-06 18:28 MainProcess)


time: 9.39 s


In [442]:
m.sourcesdf["ukdistricts"]["data"].dtypes

[cellevents:43:INFO]:starting (time=Aug-06 20:23 MainProcess)


authority      object
geometry     geometry
names         float64
registr        object
cats           object
dtype: object

time: 15 ms


In [445]:
m = UKMap()

# data sources
m.addSource('ukdistricts', df)
m.addSource('centroids', df_c)

# layers
m.addLayer("shading", type="fill", source="ukdistricts", property="names")
m.addLayer("border", type="line", source="ukdistricts")
m.addLayer("names", type="symbol", source="centroids", property="authority")

# non-mapbox features
m.title = f"Registr usage up to July 2020"
m.iframe()

[cellevents:43:INFO]:starting (time=Aug-06 20:30 MainProcess)


time: 2.81 s


In [440]:
m.layers

[cellevents:43:INFO]:starting (time=Aug-06 20:23 MainProcess)


[{'type': 'fill',
  'source': 'ukdistricts',
  'property': 'registr',
  'id': 'shading',
  'legend': [('non-user', 'rgb(141,211,199)'), ('user', 'rgb(255,255,179)')],
  'paint': {'fill-color': ['match',
    ['get', 'cats'],
    'non-user',
    'rgb(141,211,199)',
    'user',
    'rgb(255,255,179)',
    'w',
    'h',
    'i',
    't',
    'e']}},
 {'type': 'line', 'source': 'ukdistricts', 'id': 'border', 'legend': []},
 {'type': 'symbol',
  'source': 'centroids',
  'property': 'authority',
  'id': 'names',
  'legend': [],
  'paint': {'text-color': '#2a3f5f'},
  'layout': {'text-field': ['get', 'authority'], 'text-size': 10}}]

time: 0 ns


In [398]:
print(m.html())

[cellevents:43:INFO]:starting (time=Aug-06 18:02 MainProcess)


<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <title>Display a map</title>
    <meta name="viewport" content="initial-scale=1,maximum-scale=1,user-scalable=no" />
    <script src="https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.js"></script>
    <link href="https://api.mapbox.com/mapbox-gl-js/v1.11.1/mapbox-gl.css" rel="stylesheet" />
    <link href="http://localhost:8000/base.css" rel="stylesheet" />
</head>

<body>
    <div id="title">Registr usage up to July 2020</div>
    <div id="map"></div>
    <div id="rightblock">
        <div id="filter-group"><div class="filter-group"><input checked="checked" id="shading" type="checkbox"/><label for="shading">shading</label><input checked="checked" id="border" type="checkbox"/><label for="border">border</label><input checked="checked" id="names" type="checkbox"/><label for="names">names</label></div></div>
        <div id="legends"><div><div id="shading_legend"><span style="display:block; margin:5px"><span style="backg

In [493]:
with open("map2.html", "w") as f:
    f.write(m.html())

[cellevents:43:INFO]:starting (time=Aug-05 15:10 MainProcess)


time: 1.98 s
